In [1]:
import numpy as np

import ufl
from dolfinx import fem, io, mesh, plot
from ufl import ds, dx, grad, inner

from mpi4py import MPI
from petsc4py.PETSc import ScalarType

msh = mesh.create_rectangle(comm=MPI.COMM_WORLD,
                            points=((0.0, 0.0), (1.0, 1.0)), n=(8, 8),
                            cell_type=mesh.CellType.triangle,)
V = fem.FunctionSpace(msh, ("Lagrange", 1))

facets = mesh.locate_entities_boundary(msh, dim=(msh.topology.dim - 1),
                                       marker=lambda x: np.isclose(x[0], 0.0) | \
                                                        np.isclose(x[0], 1.0) | \
                                                        np.isclose(x[1], 0.0) | \
                                                        np.isclose(x[1], 1.0))

dofs = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facets)
bc = fem.dirichletbc(value=ScalarType(0), dofs=dofs, V=V)

# Define variational problem
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
x = ufl.SpatialCoordinate(msh)
f = 10 * ufl.exp(-((x[0] - 0.5) ** 2 + (x[1] - 0.5) ** 2) / 0.02)
a = inner(grad(u), grad(v)) * dx
L = inner(f, v) * dx

# Solve
problem = fem.petsc.LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [2]:
import pyvista
cells, types, x = plot.create_vtk_mesh(V)

128.0

In [18]:
x[:, 0:2]

array([[ 8.75000000e-01, -2.69278509e-18],
       [ 1.00000000e+00, -2.69278509e-18],
       [ 1.00000000e+00,  1.25000000e-01],
       [ 8.75000000e-01,  1.25000000e-01],
       [ 7.50000000e-01, -2.69278509e-18],
       [ 1.00000000e+00,  2.50000000e-01],
       [ 7.50000000e-01,  1.25000000e-01],
       [ 8.75000000e-01,  2.50000000e-01],
       [ 6.25000000e-01, -2.69278509e-18],
       [ 1.00000000e+00,  3.75000000e-01],
       [ 6.25000000e-01,  1.25000000e-01],
       [ 7.50000000e-01,  2.50000000e-01],
       [ 8.75000000e-01,  3.75000000e-01],
       [ 5.00000000e-01, -2.69278509e-18],
       [ 1.00000000e+00,  5.00000000e-01],
       [ 5.00000000e-01,  1.25000000e-01],
       [ 6.25000000e-01,  2.50000000e-01],
       [ 7.50000000e-01,  3.75000000e-01],
       [ 8.75000000e-01,  5.00000000e-01],
       [ 3.75000000e-01, -2.69278509e-18],
       [ 1.00000000e+00,  6.25000000e-01],
       [ 3.75000000e-01,  1.25000000e-01],
       [ 5.00000000e-01,  2.50000000e-01],
       [ 6.

In [14]:
128*3

384

In [20]:
pyvista.CellType.TRIANGLE

<CellType.TRIANGLE: 5>